# P0 with Exception Test Cases

#### Author: Kevin Zhou, Fanping Jiang, Meijing Li

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from IPython.display import display
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

    
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i)
    def writeln(): print('\n')
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


In [2]:
import warnings
warnings.filterwarnings("ignore")

### Scanner Tests for Exception

In [3]:
import SC
def scanString(src):
    SC.init(src); syms = []
    while SC.sym != SC.EOF:
        syms.append(('INDENT' if SC.sym == SC.INDENT else \
                     'DEDENT' if SC.sym == SC.DEDENT else \
                     'IDENT' if SC.sym == SC.IDENT else SC.sym, SC.newline))
        SC.getSym()
    return syms

### Parsing Explicit Exceptions

In [ ]:
scanString("""

program p

  if a then
    writeln()
  else
    writeln()
  if a then writeln() else writeln()
""")

In [ ]:
scanString("""

program p

  if a then
    throw
  else
    throw
  if a then writeln() else writeln()
""")

In [ ]:
scanString("""

program p

  if a then
    throw 66
  else
    throw 88
  if a then throw 99 else writeln()
""")

### Tests for Explicit Exceptions

In [4]:
# "Throw" Exception
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""")

Importing Jupyter notebook from CGwat.ipynb
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(tag $e28 (param i32))
(func $sqrt (param $x i32) (result i32)
(local $r i32)
(local $0 i32)
local.get $x
i32.const 0
i32.lt_s
if
i32.const 28
throw $e28
else
local.get $x
local.set $r
end
local.get $r)
(func $program
(local $a i32)
(local $0 i32)
i32.const 2
call $sqrt
local.set $a
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [5]:
# "Throw" Exception for testing the WebAssembly code execution

compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 28 else r := x
program equationsolver
    var a: integer
        a ← sqrt(2)
        write(a)
""", 'test.wat')

In [6]:
!wat2wasm --enable-exceptions test.wat

In [7]:
runwasm("test.wat")

<IPython.core.display.Javascript object>

In [ ]:
# "throw" and "try-catch" Exception
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 3 else r := x
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
            catch 4
                done := true
""")

In [ ]:
# "throw" and "try-catch" Exception for testing the WebAssembly code execution
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 3 else r := x
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        
        while ¬done do
            try
                a ← read(); b ← read(); c ← read()
                x, y ← quadraticsolution(a, b, c)
                write(x); write(y)
            catch 4
                done := true
""", 'test.wat')

In [6]:
!wat2wasm --enable-exceptions test.wat

In [7]:
runwasm("test.wat")

<IPython.core.display.Javascript object>

## Implicit Exception Test Cases

### Zero Division and Mod by Zero Exception

In [8]:
# Zero Division Exception
# logic: if a == 0 then throw else...
compileString("""

program DividedByZero
    var a, b: integer
        a := (34 × 5 - 170)
        b := (99 div a)
        write(a)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $a i32)
(local $b i32)
(local $0 i32)
i32.const 0
local.set $a
i32.const 99
local.get $a
i32.div_s
local.set $b
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [ ]:
# for testing the WebAssembly code execution
compileString("""

program DividedByZero
    var a, b: integer
        a := (34 × 5 - 170)
        b := (99 div a)
        write(a)
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [10]:
# Mod by 0 Exception
# logic: if a == 0 then throw else...
compileString("""

program ModByZero
    var a, b: integer
        a := (34 × 5 - 170)
        b := (99 mod a)
        write(a)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $a i32)
(local $b i32)
(local $0 i32)
i32.const 0
local.set $a
i32.const 99
local.get $a
i32.rem_s
local.set $b
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [10]:
# Mod by 0 Exception for testing the WebAssembly code execution

compileString("""

program ModByZero
    var a, b: integer
        a := (34 × 5 - 170)
        b := (99 mod a)
        write(a)
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $a i32)
(local $b i32)
(local $0 i32)
i32.const 0
local.set $a
i32.const 99
local.get $a
i32.rem_s
local.set $b
local.get $a
call $write
)
(memory 1)
(start $program)
)


In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [9]:
# the exception of this case should be handled before the code generation
compileString("""

program DividedByZero
    var b: integer
        b := (32 div 0)
        write(b)
""", 'wasmtest.wat')

Exception: integer division or modulo by zero

### Index Out of Bounds Exception

In [13]:
# logic: if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    i := 5
    write(x[i])
  
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $i i32)
(local $0 i32)
i32.const 5
local.set $i
local.get $i
i32.const 2
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
i32.load
call $write
)
(memory 1)
(start $program)
)


In [12]:
# for testing the WebAssembly code execution
compileString("""

var x: [2 .. 4] → integer
program IndexOutOfBounds
    var i: integer
    i := 5
    write(x[i])
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

In [16]:
# logic: if (i on stack < a.tp.lower) or (i on stack >= a.tp.lower + a.tp.length) then throw else ...
compileString("""

var x: [1 .. 5] → integer
program IndexOutOfBounds
    var i: integer
    i := -1
    write(x[i])
  
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $program
(local $i i32)
(local $0 i32)
i32.const -1
local.set $i
local.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 0
i32.add
i32.load
call $write
)
(memory 1)
(start $program)
)


In [15]:
# for testing the WebAssembly code execution
compileString("""

var x: [1 .. 5] → integer
program IndexOutOfBounds
    var i: integer
    i := -1
    write(x[i])
  
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")

### Nested `try catch` Exception

In [18]:
# "throw" and "try-catch" Exception with try-catch inside a try-catch
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 3 else r := x
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        
        while ¬done do
            try
                try
                    a ← read(); b ← read(); c ← read()
                    x, y ← quadraticsolution(a, b, c)
                    write(x); write(y)
                catch 5
                    done := true
            catch 6
                done := false
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(tag $e6 (param i32))
(tag $e5 (param i32))
(tag $e3 (param i32))
(func $sqrt (param $x i32) (result i32)
(local $r i32)
(local $0 i32)
local.get $x
i32.const 0
i32.lt_s
if
i32.const 3
throw $e3
else
local.get $x
local.set $r
end
local.get $r)
(func $quadraticsolution (param $a i32) (param $b i32) (param $c i32) (result i32) (result i32)
(local $x i32)
(local $y i32)
(local $d i32)
(local $0 i32)
local.get $a
local.get $a
i32.mul
i32.const 4
local.get $a
i32.mul
local.get $c
i32.mul
i32.sub
call $sqrt
local.set $d
local.get $b
i32.const -1
i32.mul
local.get $d
i32.add
i32.const 2
local.get $a
i32.mul
i32.div_s
local.get $b
i32.const -1
i32.mul
local.get $d
i32.sub
i32.const 2
local.get $a
i32.mul
i32.div_s
local.set $y
local.set $x
local.get $x
local.get $y)
(func $program
(local $a i32)
(local $b i32)
(local $c i32)
(local $x i32)
(loc

In [ ]:
# "throw" and "try-catch" Exception with try-catch inside a try-catch
# for testing the WebAssembly code execution
compileString("""
procedure sqrt(x: integer) → (r: integer)
    if x < 0 then throw 3 else r := x
procedure quadraticsolution(a, b, c: integer) → (x, y: integer)
    var d: integer
        d ← sqrt(a × a - 4 × a × c)
        x, y := (- b + d) div (2 × a), (- b - d) div (2 × a)
program equationsolver
    var a, b, c, x, y: integer
    var done: boolean
        done := false
        
        while ¬done do
            try
                try
                    a ← read(); b ← read(); c ← read()
                    x, y ← quadraticsolution(a, b, c)
                    write(x); write(y)
                catch 5
                    done := true
            catch 6
                done := false
""", 'wasmtest.wat')

In [ ]:
!wat2wasm --enable-exceptions wasmtest.wat

In [ ]:
runwasm("wasmtest.wat")